In [2]:
import pandas as pd
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

import matplotlib.pyplot as plt

import scipy.sparse as sparse
from implicit.evaluation import train_test_split, precision_at_k, mean_average_precision_at_k
import random

import implicit

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

%watermark -a 'Rebecca Vickery' -d -t -v -p scipy,numpy,pandas,matplotlib,implicit

UsageError: Line magic function `%watermark` not found.


In [17]:
data = pd.read_csv('~/Downloads/train.csv')
data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [18]:
# msn = user_id
# song_id = item_id

data['song_id'].value_counts()

reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=    13973
T86YHdD4C9JSc274b1IlMkLuNdz4BQRB50fWWE7hx9g=    13293
wBTWuHbjdjxnG1lQcbqnK4FddV24rUhuyrYLd9c/hmk=    13079
FynUyq0+drmIARmK1JZ/qcjNZ7DKkqTY6/0O0lTzNUI=    12855
PgRtmmESVNtWjoZHO5a1r21vIz9sVZmcJJpFCbRa1LI=    12004
                                                ...  
KI36QrxD5dy0k3pu9QU+qMcHswu860t0sjiRxy4qa7I=        1
rW9pOAbWXuDhJ14/aAMQfKy1cJ+kJjRrN53eXBNSJUk=        1
9VQ0a1R+MZjq44nZi+EvvJlFpuH/AYolB0mx/DfPsJ8=        1
/0WTj9gE/v8GYJlsaxzi9R+4NGIvbOE7eDgaI8lNX5U=        1
bF7yMUs6R93ZwE8grDrI9DsRDfExtuf+BUyOwffloNo=        1
Name: song_id, Length: 359966, dtype: int64

In [19]:
data['customer_id'] = data['msno']
data = data.drop('msno', axis=1)
data.columns

Index(['song_id', 'source_system_tab', 'source_screen_name', 'source_type',
       'target', 'customer_id'],
      dtype='object')

In [20]:
data['customer_id'] = data['customer_id'].astype('category').cat.codes
data['song_id_num'] = data['song_id'].astype('category').cat.codes

item_lookup = data[['song_id_num', 'song_id']].drop_duplicates()
item_lookup['song_id_num'] = item_lookup.song_id.astype(str)

In [22]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
validation = data[~msk]

In [23]:
print(data.customer_id.nunique())
print(data.song_id_num.nunique())

30755
359966


In [24]:
# Create item customer and customer item matrices

sparse_item_customer_train = sparse.csr_matrix((train['target'].astype(float), 
                                                     (train['song_id_num'], train['customer_id'])))
sparse_customer_item_train = sparse.csr_matrix((train['target'].astype(float), 
                                                (train['customer_id'], train['song_id_num'])))

In [25]:
# Initialize the als model and fit it using the sparse item-customer matrix
model = implicit.als.AlternatingLeastSquares(factors=20, 
                                             regularization=0.1, 
                                             iterations=20, 
                                             calculate_training_loss=True)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_customer_train * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

In [27]:
sparse_item_customer_train = sparse.csr_matrix((validation['target'].astype(float), 
                                                     (validation['song_id_num'], validation['customer_id'])))
sparse_customer_item_train = sparse.csr_matrix((validation['target'].astype(float), 
                                                (validation['customer_id'], validation['song_id_num'])))

In [ ]:
recommendations_validation = model.recommend_all(user_items=sparse_customer_item_validation_pr,
                                  N=10, 
                                  recalculate_user=False, 
                                  filter_already_liked_items=False
                                      )

#recommeded_validation_df = validation[['email', 'email_id']]

recommeded_validation_df = pd.DataFrame(validation)

recommeded_validation_df.head()


destination_processed_data = validation[['destination_id', 'destination']].drop_duplicates().copy().reset_index(drop=True)
destination_processed_data = destination_processed_data.set_index('destination_id')
destination_dict = pd.Series(destination_processed_data.destination).to_dict()

# create a dictionary of customer interger id's and customer id' from processed data
customer_processed_data = df_val_predict[['n_customer_id', 'customer_id']].drop_duplicates().copy().reset_index(drop=True)
customer_processed_data = customer_processed_data.set_index('n_customer_id')
customer_dict = pd.Series(customer_processed_data.customer_id).to_dict()

# replace recommended product id's with product codes
recommeded_validation_df = recommeded_validation_df.replace(destination_dict)

# replace customer_nid index with customer_id
recommeded_validation_df.index = recommeded_validation_df.index.map(customer_dict)

# convert customer_id from index to column and create new index
recommeded_validation_df = recommeded_validation_df.rename_axis('customer_id').reset_index()
recommeded_validation_df = recommeded_validation_df.dropna()
recommeded_validation_df.head()